In [16]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth
import plotly.express as px

In [3]:
PASSWORD_NEO4J = 'passwordneo4j'

In [4]:
driver = GraphDatabase.driver(
  "bolt://localhost:7687",
  auth=basic_auth("neo4j", PASSWORD_NEO4J))

In [28]:
def get_all_road_points(tx, x_1, y_1, x_2, y_2):

    query = " WITH $x_1 as x_1, $x_2 as x_2, $y_1 as y_1, $y_2 as y_2 \
            MATCH (p:ROAD_POINT) \
            WHERE p.x > x_1 AND p.x < x_2 \
            AND p.y > y_1 AND p.y < y_2 \
            RETURN p.x AS x, p.y AS y"
            

    result = tx.run(query, x_1=x_1, y_1=y_1, x_2=x_2, y_2=y_2)
    return result.data()


In [33]:
def get_all_national_road_points(tx, x_1, y_1, x_2, y_2):

    query = " WITH $x_1 as x_1, $x_2 as x_2, $y_1 as y_1, $y_2 as y_2 \
            MATCH (p:NATIONAL_ROAD_POINT) \
            WHERE p.x > x_1 AND p.x < x_2 \
            AND p.y > y_1 AND p.y < y_2 \
            RETURN p.x AS x, p.y AS y"
            

    result = tx.run(query, x_1=x_1, y_1=y_1, x_2=x_2, y_2=y_2)
    return result.data()


In [40]:
x_1 = -5.5
y_1 = 47.2
x_2 = -1.2
y_2 = 49

In [41]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(get_all_road_points, x_1=x_1, y_1=y_1, x_2=x_2, y_2=y_2)

driver.close()

df = pd.DataFrame(result)

In [42]:
df['label'] = 'ROAD_POINT'

In [43]:
df

,x,y,label
0,-4.780139,48.437121,ROAD_POINT
1,-4.777558,48.359313,ROAD_POINT
2,-4.777093,48.421995,ROAD_POINT
3,-4.774678,48.358647,ROAD_POINT
4,-4.771587,48.360735,ROAD_POINT
...,...,...,...
46780,-1.200294,48.048886,ROAD_POINT
46781,-1.200269,47.651706,ROAD_POINT
46782,-1.200174,47.386663,ROAD_POINT
46783,-1.200144,47.371392,ROAD_POINT


In [44]:
with driver.session() as session:

    count_list = []

    result = session.execute_write(get_all_national_road_points, x_1=x_1, y_1=y_1, x_2=x_2, y_2=y_2)

driver.close()

df_2 = pd.DataFrame(result)

In [45]:
df_2['label'] = 'NATIONAL_ROAD_POINT'

In [46]:
df_all = pd.concat([df, df_2], axis=0)

In [47]:
df_all

,x,y,label
0,-4.780139,48.437121,ROAD_POINT
1,-4.777558,48.359313,ROAD_POINT
2,-4.777093,48.421995,ROAD_POINT
3,-4.774678,48.358647,ROAD_POINT
4,-4.771587,48.360735,ROAD_POINT
...,...,...,...
341,-2.205760,47.304970,NATIONAL_ROAD_POINT
342,-2.417210,48.406220,NATIONAL_ROAD_POINT
343,-4.444490,48.435280,NATIONAL_ROAD_POINT
344,-1.688530,47.256750,NATIONAL_ROAD_POINT


In [48]:
fig = px.scatter_mapbox(df_all, lat='y', lon='x', color='label', mapbox_style='open-street-map')
fig.show()